# Классификация текстов: спам-фильтр для SMS

В этом задании вам предстоит взять открытый датасет с SMS-сообщениями, размеченными на спам ("spam") и не спам ("ham"), построить на нем классификатор текстов на эти два класса, оценить его качество с помощью кросс-валидации, протестировать его работу на отдельных примерах, и посмотреть, что будет происходить с качеством, если менять параметры вашей модели.

In [49]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score

Загрузите датасет. Считайте датасет в Python (можете сразу грузить все в память, выборка небольшая), выясните, что используется в качестве разделителей и как проставляются метки классов.

In [8]:
data = pd.read_csv('smsspamcollection/SMSSpamCollection.txt', sep='\t', header=None, names=['tag', 'text'])
data.head()

,tag,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [9]:
data.tag.value_counts()

ham     4825
spam     747
Name: tag, dtype: int64

Подготовьте для дальнейшей работы два списка: список текстов в порядке их следования в датасете и список соответствующих им меток классов. В качестве метки класса используйте 1 для спама и 0 для "не спама".

In [13]:
text = data.text.values
labels = [1 if x =='spam' else 0 for x in data.tag.values]

Используя sklearn.feature_extraction.text.CountVectorizer со стандартными настройками, получите из списка текстов матрицу признаков X.

In [18]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(text)

In [25]:
print(vectorizer.get_feature_names()[1111])

anniversary


Оцените качество классификации текстов с помощью LogisticRegression() с параметрами по умолчанию, используя sklearn.cross_validation.cross_val_score и посчитав среднее арифметическое качества на отдельных fold'ах. Установите random_state=2. Параметр cv задайте равным 10. В качестве метрики качества используйте f1-меру. Получившееся качество - один из ответов, которые потребуются при сдаче задания. Ответ округлить до 1 знака после запятой.

In [29]:
scores = cross_val_score(LogisticRegression(random_state=2), X, labels, cv=10, scoring='f1')

C:\Users\Alexey\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [31]:
scores.mean()

0.9326402983610631

А теперь обучите классификатор на всей выборке и спрогнозируйте с его помощью класс для следующих сообщений:

In [32]:
messages = [
    "FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
    "FreeMsg: Txt: claim your reward of 3 hours talk time",
    "Have you visited the last lecture on physics?",
    "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
    "Only 99$"]

In [34]:
lr = LogisticRegression(random_state=2)
lr.fit(X, labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=2, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [37]:
X_new = vectorizer.transform(messages)
lr.predict(X_new)

array([1, 1, 0, 0, 0])

Задайте в CountVectorizer параметр ngram_range=(2,2), затем ngram_range=(3,3), затем ngram_range=(1,3). Во всех трех случаях измерьте получившееся в кросс-валидации значение f1-меры, округлите до второго знака после точки, и выпишете результаты через пробел в том же порядке. В данном эксперименте мы пробовали добавлять в признаки n-граммы для разных диапазонов n - только биграммы, только триграммы, и, наконец, все вместе - униграммы, биграммы и триграммы. Обратите внимание, что статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.

In [39]:
vect = CountVectorizer(ngram_range=(2, 2))
objects = vect.fit_transform(text)
scores = cross_val_score(LogisticRegression(random_state=2), objects, labels, scoring='f1', cv=10)
scores.mean()

C:\Users\Alexey\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8224220664187133

In [40]:
vect = CountVectorizer(ngram_range=(3, 3))
objects = vect.fit_transform(text)
scores = cross_val_score(LogisticRegression(random_state=2), objects, labels, scoring='f1', cv=10)
scores.mean()

C:\Users\Alexey\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7250161555467377

In [44]:
vect = CountVectorizer(ngram_range=(1, 3))
objects = vect.fit_transform(text)
scores = cross_val_score(LogisticRegression(random_state=2), objects, labels, scoring='f1', cv=10)
scores.mean()

C:\Users\Alexey\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9251382558648837

Повторите аналогичный эксперимент, используя вместо логистической регрессии MultinomialNB(). Обратите внимание, насколько сильнее (по сравнению с линейным классификатором) наивный Байес страдает от нехватки статистики по биграммам и триграммам.

In [47]:
vect = CountVectorizer(ngram_range=(2, 2))
objects = vect.fit_transform(text)
scores = cross_val_score(MultinomialNB(), objects, labels, scoring='f1', cv=10)
print(scores.mean())

vect = CountVectorizer(ngram_range=(3, 3))
objects = vect.fit_transform(text)
scores = cross_val_score(MultinomialNB(), objects, labels, scoring='f1', cv=10)
print(scores.mean())

vect = CountVectorizer(ngram_range=(1, 3))
objects = vect.fit_transform(text)
scores = cross_val_score(MultinomialNB(), objects, labels, scoring='f1', cv=10)
print(scores.mean())

0.6455015177985443
0.37871948524573595
0.8884859656061002


Попробуйте использовать в логистической регрессии в качестве признаков Tf*idf из TfidfVectorizer на униграммах. Повысилось или понизилось качество на кросс-валидации по сравнению с CountVectorizer на униграммах? (напишите в файле с ответом 1, если повысилось, -1, если понизилось, и 0, если изменилось не более чем на 0.01). Обратите внимание, что результат перехода к tf*idf не всегда будет таким - если вы наблюдаете какое-то явление на одном датасете, не надо сразу же его обобщать на любые данные.

In [48]:
tfidf_vectorizer = TfidfVectorizer()
X = tfidf_vectorizer.fit_transform(text)
scores = cross_val_score(LogisticRegression(random_state=2), X, labels, scoring='f1', cv=10)
print(scores.mean())

0.8528599554172456


C:\Users\Alexey\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
